In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade setuptools pip --user
!pip install --ignore-installed PyYAML
!pip install Pillow

!pip install nvidia-pyindex
!pip install --upgrade nvidia-tensorrt
!pip install pycuda

!pip install protobuf<4.21.3
!pip install onnxruntime-gpu
!pip install onnx>=1.9.0
!pip install onnx-simplifier>=0.3.6 --user

In [3]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.0.1+cu118 


In [4]:
!nvidia-smi

Sun Jun 25 05:34:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!# Download YOLOv7 code
!git clone -b u7 https://github.com/WongKinYiu/yolov7
%cd yolov7/seg
!ls

In [15]:
# inference for base model of yolov7 (pytorch model)
!python segment/predict.py --weights /content/drive/MyDrive/in-progress-anomalib/optimization/models/best.pt --conf 0.10 --img-size 1024 --source /content/drive/MyDrive/in-progress-anomalib/optimization/yolov7_onnx_tesnorrt/Pic_2023_05_25_141948_31_0_1.png

segment/predict: weights=['/content/drive/MyDrive/in-progress-anomalib/optimization/models/best.pt'], source=/content/drive/MyDrive/in-progress-anomalib/optimization/yolov7_onnx_tesnorrt/Pic_2023_05_25_141948_31_0_1.png, data=data/coco128.yaml, imgsz=[1024, 1024], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2023-6-22 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 325 layers, 37842476 parameters, 0 gradients
image 1/1 /content/drive/MyDrive/in-progress-anomalib/optimization/yolov7_onnx_tesnorrt/Pic_2023_05_25_141948_31_0_1.png: 1024x1024 2 defects, 106.0ms
Speed: 1.1ms pre-process, 106.0ms inference, 11.0ms NMS per image at shape (1, 3, 

In [16]:
# convert pytorch model to onnx model
!python export.py --weights /content/drive/MyDrive/in-progress-anomalib/optimization/models/best.pt  --simplify --topk-all 100 --iou-thres 0.65 --conf-thres 0.10 --img-size 1024 1024 --include 'onnx'


export: data=data/coco128.yaml, weights=['/content/drive/MyDrive/in-progress-anomalib/optimization/models/best.pt'], imgsz=[1024, 1024], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.65, conf_thres=0.1, include=['onnx']
YOLOv5 🚀 2023-6-22 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 325 layers, 37842476 parameters, 0 gradients

PyTorch: starting from /content/drive/MyDrive/in-progress-anomalib/optimization/models/best.pt with output shape (1, 64512, 38) (73.1 MB)
requirements: onnx not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 23.6 MB/s eta 0:00:00
  Attempt

In [17]:
# Download ONNX to TensorRT converter
%cd ../
!git clone https://github.com/Linaom1214/tensorrt-python.git

/content/drive/MyDrive/in-progress-anomalib/optimization/yolov7/yolov7
Cloning into 'tensorrt-python'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 323 (delta 57), reused 41 (delta 41), pack-reused 239
Receiving objects: 100% (323/323), 132.55 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
%cd tensorrt-python
!ls

/content/yolov7/yolov7/yolov7/tensorrt-python
cpp  export.py	image_batch.py	README_CN.md  README.md  src  trt.py  utils


In [19]:
# Export TensorRT-engine model
!python export.py -o /content/drive/MyDrive/in-progress-anomalib/optimization/models/best.onnx -e ./yolov7-tiny-nms-conf-new.trt -p fp16

Namespace(onnx='/content/drive/MyDrive/in-progress-anomalib/optimization/models/best.onnx', engine='./yolov7-tiny-nms-conf-new.trt', precision='fp16', verbose=False, workspace=1, calib_input=None, calib_cache='./calibration.cache', calib_num_images=5000, calib_batch_size=8, end2end=False, conf_thres=0.4, iou_thres=0.5, max_det=100, v8=False)
[06/25/2023-05:51:14] [TRT] [I] [MemUsageChange] Init CUDA: CPU +340, GPU +0, now: CPU 363, GPU 173 (MiB)
[06/25/2023-05:51:21] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +732, GPU +172, now: CPU 1172, GPU 345 (MiB)
[06/25/2023-05:51:21] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[06/25/2023-05:51:21] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not 

In [ ]:
import cv2
import torch
import random
import time
import numpy as np
import tensorrt as trt
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple

In [ ]:
w = '/content/yolov7/yolov7/yolov7/tensorrt-python/yolov7-tiny-nms.trt'
device = torch.device('cuda:0')
img = cv2.imread('/content/drive/MyDrive/in-progress-anomalib/optimization/yolov7_onnx_tesnorrt/Pic_2023_05_25_141948_31_0_1.png')

In [ ]:
# Infer TensorRT Engine
Binding = namedtuple('Binding', ('name', 'dtype', 'shape', 'data', 'ptr'))
logger = trt.Logger(trt.Logger.INFO)
trt.init_libnvinfer_plugins(logger, namespace="")
with open(w, 'rb') as f, trt.Runtime(logger) as runtime:
    model = runtime.deserialize_cuda_engine(f.read())
bindings = OrderedDict()
for index in range(model.num_bindings):
    name = model.get_binding_name(index)
    dtype = trt.nptype(model.get_binding_dtype(index))
    shape = tuple(model.get_binding_shape(index))
    data = torch.from_numpy(np.empty(shape, dtype=np.dtype(dtype))).to(device)
    bindings[name] = Binding(name, dtype, shape, data, int(data.data_ptr()))
binding_addrs = OrderedDict((n, d.ptr) for n, d in bindings.items())
context = model.create_execution_context()


def letterbox(im, new_shape=(1024, 1024), color=(114, 114, 114), auto=True, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, r, (dw, dh)



<ipython-input-152-66dc47a45a45>:9: DeprecationWarning: Use get_tensor_name instead.
  name = model.get_binding_name(index)
<ipython-input-152-66dc47a45a45>:10: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(model.get_binding_dtype(index))
<ipython-input-152-66dc47a45a45>:11: DeprecationWarning: Use get_tensor_shape instead.
  shape = tuple(model.get_binding_shape(index))


In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image = img.copy()
image, ratio, dwdh = letterbox(image, auto=False)
image = image.transpose((2, 0, 1))
image = np.expand_dims(image, 0)
image = np.ascontiguousarray(image)

im = image.astype(np.float32)
im.shape

(1, 3, 1024, 1024)

In [ ]:
im = torch.from_numpy(im).to(device)
im/=255
im.shape

# warmup for 10 times
for _ in range(10):
    tmp = torch.randn(1,3,1024,1024).to(device)
    binding_addrs['images'] = int(tmp.data_ptr())
    context.execute_v2(list(binding_addrs.values()))

start = time.perf_counter()
binding_addrs['images'] = int(im.data_ptr())
context.execute_v2(list(binding_addrs.values()))
print(f'Cost {time.perf_counter()-start} s')

out = bindings['output'].data


In [ ]:
bindings

OrderedDict([('images',
              Binding(name='images', dtype=<class 'numpy.float32'>, shape=(1, 3, 1024, 1024), data=tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                 